In [1]:
%matplotlib inline
import cv2
from numpy import floor
from scipy.ndimage.filters import gaussian_filter1d
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage.filters import maximum_filter
from scipy.ndimage.interpolation import map_coordinates
from scipy.spatial.distance import cdist
import math
import matplotlib.pyplot as plt
import numpy as np



In [2]:

def gauss_derivative_kernels(size, sizey=None):
    """
        Function to get gaussian deritive kernel
    :param size: size of kernel
    :param sizey: get y size
    :return: 
    """
    size = int(size)
    if not sizey:
        sizey = size
    else:
        sizey = int(sizey)
    y, x = mgrid[-size:size+1, -sizey:sizey+1]
    #x and y derivatives of a 2D gaussian with standard dev half of size
    # (ignore scale factor)
    gx = - x * exp(-(x**2/float((0.5*size)**2)+y**2/float((0.5*sizey)**2))) 
    gy = - y * exp(-(x**2/float((0.5*size)**2)+y**2/float((0.5*sizey)**2))) 
    return gx,gy

In [3]:
def normalize(img):
    '''
     Function to normalize an input array to 0-1 
    '''
    img_min = img.min()
    img_max = img.max()
    return (img - img_min) / (img_max - img_min)

def my_normalize(matches):
    mean_num = np.mean(matches,axis=0)
    m_off = np.eye(3,dtype=float)
    m_off[0][2],m_off[1][2] = -mean_num[0],-mean_num[1]
    m_scale= np.eye(3,dtype=float)
    m_scale[0, 0] = 1.0 / max(abs(matches[:, 0]))
    m_scale[1, 1] = 1.0 / max(abs(matches[:, 1]))
    coor_trans = np.matmul(m_scale,m_off)
    noralized_match = np.transpose(np.matmul(coor_trans,
                                             np.transpose(matches)))
    
    return coor_trans,noralized_match


In [4]:
"""
Function for harris edge detections
"""
edge = 16
# def harris(filename, min_distance = 10, threshold = 0.1):
#     """
#     filename: Path of image file
#     threshold: (optional)Threshold for corner detection
#     min_distance : (optional)Minimum number of pixels separating 
#      corners and image boundary
#     """
#     im = np.array(Image.open(filename).convert("L"))
#     harrisim = compute_harris_response(im)
#     filtered_coords = get_harris_points(harrisim,min_distance, threshold)
#     plot_harris_points(im, filtered_coords)
def harris_cornor_detector(img, num_of_points =500):    
    direction_x = gaussian_filter1d(
        gaussian_filter1d(img.astype(np.float32), 1.0, 0, 0), 1.0, 1, 1
    )
    direction_y = gaussian_filter1d(gaussian_filter1d(img.astype(np.float32),
            1.0, 1, 0), 1.0, 0, 1)
    h = (gaussian_filter(direction_x ** 2, 1.5, 0) * gaussian_filter(direction_y ** 2, 1.5, 0) - gaussian_filter(direction_x * direction_y, 1.5, 0)**2) \
        / (gaussian_filter(direction_x**2, 1.5, 0) + gaussian_filter(direction_y**2, 1.5, 0) + 1e-8)
    h[:edge, :], h[-edge:, :], h[:, :edge], h[:, -edge:] = 0, 0, 0, 0
    mask = h == maximum_filter(h,(8,8))
    h = h * mask
    final_pairs = np.argsort(h.flatten())[::-1][:num_of_points ]
    return np.vstack((final_pairs % img.shape[0:2][1], final_pairs / img.shape[0:2][1], h.flatten()[final_pairs])).transpose()


In [5]:
neighbor_hood=8 # neighbor numbers


def extract_decriptor(img, harris ):
    """
    Function to extract descriptor 
    :param img:  input image
    :param harris: harris cornor 
    :return: descriptor from detected points
    """
    # Change number here to change the scale. 4 is the optimal amount 
    y, x = 4 * np.mgrid[-neighbor_hood:neighbor_hood+1, -neighbor_hood:neighbor_hood+1]
    descriptor = np.zeros((2 * neighbor_hood + 1, 2 * neighbor_hood + 1, harris.shape[0]), dtype=float)
    for i in range(harris.shape[0]):
        tmp_points = map_coordinates(img,[harris[i,1] + y, harris[i,0] + x], prefilter=False)
        # normalize the descriptor
        descriptor[..., i] = (tmp_points - tmp_points.mean()) / tmp_points.std()
    return descriptor

In [6]:
def get_match_from_dist(d1, d2,threshold = 0.7):
    """
        Function to get matches from descriptor according to cdist
    """
    _, width, n = d1.shape[0:3]
    #  p1 = np.transpose(np.matrix([correspondence[0].item(0), correspondence[0].item(1), 1]))
    # estimatep2 = np.dot(h, p1)
    # estimatep2 = (1/estimatep2.item(2))*estimatep2
    distance = cdist((d1.reshape((width**2, n))).T, (d2.reshape((width**2, n))).T)
    best_current = np.argsort(distance, 1)[:, 0]
    # calculate the ration compared to best matches
    ratio = distance[np.r_[0:n], best_current] / distance[np.r_[0:n], np.argsort(distance, 1)[:, 1]].mean()
    return np.hstack([np.argwhere(ratio < threshold), best_current[np.argwhere(ratio < threshold)]]).astype(int)


In [7]:

def get_harris_points(harrisim, min_distance=10, threshold=0.1):
    """ return corners from a Harris response image
        min_distance is the minimum nbr of pixels separating 
        corners and image boundary"""
    #find top corner candidates above a threshold
    corner_threshold = max(harrisim.ravel()) * threshold
    harrisim_t = (harrisim > corner_threshold) * 1    
    #get coordinates of candidates
    candidates = harrisim_t.nonzero()
    coords = [ (candidates[0][c],candidates[1][c]) for c in range(len(candidates[0]))]
    #...and their values
    candidate_values = [harrisim[c[0]][c[1]] for c in coords]    
    #sort candidates
    index = argsort(candidate_values)   
    #store allowed point locations in array
    allowed_locations = zeros(harrisim.shape)
    allowed_locations[min_distance:-min_distance,min_distance:-min_distance] = 1   
    #select the best points taking min_distance into account
    filtered_coords = []
    for i in index:
        if allowed_locations[coords[i][0]][coords[i][1]] == 1:
            filtered_coords.append(coords[i])
            allowed_locations[(coords[i][0]-min_distance):(coords[i][0]+min_distance),
                (coords[i][1]-min_distance):(coords[i][1]+min_distance)] = 0               
    return filtered_coords





In [8]:
def homo_coor(points_list):
    """
    :param points_list: 3*N points list
    :return: homogeneous coordinates
    """
     # n_outliers = 100
     #    all_idxs = numpy.arange( A_noisy.shape[0] )
     #    numpy.random.shuffle(all_idxs)
     #    outlier_idxs = all_idxs[:n_outliers]
           # if 3*N
    if points_list.shape[0] == 3:
        homo_coordinates = np.zeros_like(points_list)
        for i in range(3):
            homo_coordinates[i, :] = points_list[i, :] / points_list[2, :]
        # divide the last row
        #     non_outlier_idxs = all_idxs[n_outliers:]
        # A_noisy[outlier_idxs] =  20*numpy.random.random((n_outliers,n_inputs) )
        # B_noisy[outlier_idxs] = 50*numpy.random.normal(size=(n_outliers,n_outputs) )
    # if 2*N, append np.ones and return 
    elif points_list.shape[0] == 2: 
        homo_coordinates = np.vstack((points_list, 
                         np.ones((1, points_list.shape[1]),dtype=points_list.dtype)))
    
    return homo_coordinates



In [9]:


def cal_homo_Matrix(point1, point2):
    """
    get homography based on two points
    :param point1: point1
    :param point2: point2
    :return: homograph of an image
    """
    A = np.matrix(np.zeros((point1.shape[0]*2, 8), dtype=float), dtype=float)
    for i in range(0, A.shape[0]):
        if i % 2 == 0:
            A[i,0] = point1[i//2,0]
            A[i,1] = point1[i//2,1]
            A[i,2] = 1
            A[i,6] = -point2[i//2,0] * point1[i//2,0]
            A[i,7] = -point2[i//2,0] * point1[i//2,1]
        else:
            A[i,3] = point1[i//2,0]
            A[i,4] = point1[i//2,1]
            A[i,5] = 1
            A[i,6] = -point2[i//2,1] * point1[i//2,0]
            A[i,7] = -point2[i//2,1] * point1[i//2,1]
    
    # get flattened b
    b = point2.flatten().reshape(point2.flatten().shape[1], 1).astype(float)
    # all_data = numpy.hstack( (A_noisy,B_noisy) )
    # input_columns = range(n_inputs) # the first columns of the array
    # output_columns = [n_inputs+i for i in range(n_outputs)] # the last columns of the array
    # Calculating  A * x = b
    x = np.array(np.linalg.lstsq(A,b)[0])
    
    x = np.append(x,np.matrix(1)).reshape((3,3))
    return x
   

In [10]:

def cal_cornor(homography, img_path):
    """
    get cornors according to homograph
    :param homography: 
    :param img_path: 
    :return: 
    """
    mid_point = None
    List_C = []
    for i in range(len(img_path)):
        height, width = cv2.imread(img_path[i]).shape[0:2]
        List_C.append(homo_coor(np.dot(homography[i], 
                                    homo_coor(np.array([[0, width, width, 0], [0, 0, height, height]], dtype=float)))).astype(int))
        # if i is half, then midpoint is last of C
        # A = numpy.vstack([data[:,i] for i in self.input_columns]).T
        # B = numpy.vstack([data[:,i] for i in self.output_columns]).T
        # x,resids,rank,s = numpy.linalg.lstsq(A,B)
        if i == len(img_path)/2:
            mid_point = List_C[-1]
    w_list1 = []
    w_list2 = []
    h_list1 = []
    h_list2 = []
    for i in range(len(List_C)):
        # A = numpy.vstack([data[:,i] for i in self.input_columns]).T
        # B = numpy.vstack([data[:,i] for i in self.output_columns]).T
        # B_fit = scipy.dot(A,model)
        w_list1 += [np.min(List_C[i][0, :])]
        h_list1 += [np.min(List_C[i][1, :])]
        h_list2 += [np.max(List_C[i][1, :])]
        w_list2 += [np.max(List_C[i][0, :])]
        
    result = np.array([(min(w_list1), min(h_list1)), (max(w_list2), max(h_list2))]), mid_point
    return result

In [11]:
def cal_middle_homo(homo_Matrix,img_size):
    """
    :param homo: 
    :return: 
    """
    middle_Homo = []
    # cal middle one's homo matrix 
    homo_Matrix[1] = homo_Matrix[0] * homo_Matrix[1]
    for i in range(len(homo_Matrix)):
        middle_Homo.append(np.linalg.inv(homo_Matrix[img_size//2]) * homo_Matrix[i])
    return middle_Homo

In [12]:

 # def plot_plane(a, b, c, d):
 #        xx, yy = np.mgrid[:10, :10]
 #        return xx, yy, (-d - a * xx - b * yy) / c
 # 
 #    n = 100
 #    max_iterations = 100
 #    goal_inliers = n * 0.3


def ransac(Matches, tolerance=0.5, iteration=100):
    """
    
    :param Matches: matches selected
    :param tolerance: tolerance for model 
    :param iteration: max iteration number
    :return: homo_matrix
    """
    # counter for iteration cnt
    count = 0
    # record last best model info
    last_consensus_cnt = 0
    last_homo,last_count = None, None
    print("Matches shape",Matches.shape)
    
    while count < iteration:
        sample_points = np.copy(Matches)
        tmp_Matches = np.matrix(np.copy(Matches))
       
        #      try ransac for inliers
        # m, b = run_ransac(xyzs, estimate, lambda x, y: is_inlier(x, y, 0.01), 3, goal_inliers, max_iterations)
        # a, b, c, d = m
        # xx, yy, zz = plot_plane(a, b, c, d)
        # ax.plot_surface(xx, yy, zz, color=(0, 1, 0, 0.5))

        # Gets a random set of points on RANSAC
        np.random.shuffle(sample_points)

        # get 4 sample points
        sample_points = np.matrix(sample_points)[0:4]
        # get homo matrix based on this two points
        homo_matrix = cal_homo_Matrix(sample_points[:,0:2], sample_points[:,2:4])
        residual_tmp = np.power(np.array(np.array(homo_coor((homo_matrix * homo_coor(tmp_Matches[:,0:2].transpose())))[0:2,:]) 
                                  - tmp_Matches[:,2:].transpose()),2)
        # cal the residual of this model
        residual = np.sqrt(residual_tmp.sum(0))
        less_than_tol_cnt = (residual < tolerance).sum()
        # if new model is better than model before
        
            #      best_ic = 0
            # best_model = None
            # random.seed(random_seed)
            # # random.sample cannot deal with "data" being a numpy array
            # data = list(data)
        
        if last_consensus_cnt < less_than_tol_cnt  :
            # update homo
            last_homo = homo_matrix
            last_consensus_cnt = (residual < tolerance).sum()
            last_count = np.argwhere(residual < tolerance)
            
            Matrix_P = float(last_consensus_cnt) / Matches.shape[0]
           
            # update iteration number according to Matrix P
            itertation = math.log(0.05) * (1/math.log(1-(np.power(Matrix_P,4))))
            
        count += 1
    print("number of homo inliers :", last_consensus_cnt)
    print("residual:", np.sum(residual)/residual.size**2)
    return last_homo


In [13]:
def image_transformation(image, middle, Cornors):
    """
     transorm image based on homography
    :param im: image 
    :param middle: middle points
    :param Cornors: Cornor
    :return: 
    """
    tmp_y, tmp_x = np.mgrid[Cornors[0, 1]:Cornors[1, 1],
                   Cornors[0, 0]:Cornors[1, 0]]
    
    height = Cornors[1, 1] - Cornors[0, 1]
    width = Cornors[1, 0] - Cornors[0, 0] 
    # change to homo coordinate
    #    p1 = np.matrix([corr.item(0), corr.item(1), 1])
    #     p2 = np.matrix([corr.item(2), corr.item(3), 1])
    # 
    A = homo_coor(np.dot(np.linalg.inv(middle), homo_coor(np.vstack((tmp_x.flatten(), tmp_y.flatten())))))
    x_a = A[0,:].reshape((height, width))
    y_a = A[1,:].reshape((height, width))
    #     a2 = [0, 0, 0, -p2.item(2) * p1.item(0), -p2.item(2) * p1.item(1), -p2.item(2) * p1.item(2),
    #           p2.item(1) * p1.item(0), p2.item(1) * p1.item(1), p2.item(1) * p1.item(2)]
    #     a1 = [-p2.item(2) * p1.item(0), -p2.item(2) * p1.item(1), -p2.item(2) * p1.item(2), 0, 0, 0,
    #           p2.item(0) * p1.item(0), p2.item(0) * p1.item(1), p2.item(0) * p1.item(2)]
    
    result = np.zeros((height, width, image.shape[2]), dtype=image.dtype)
    if image.ndim == 3:
        for dimension in range(image.shape[2]):
            #need broadcast here
            result[..., dimension] = map_coordinates(image[..., dimension], [y_a, x_a])
    else:
        result = map_coordinates(image, [y_a, x_a])
        
    # matrixA = np.matrix(aList)
    # 
    # #svd composition
    # u, s, v = np.linalg.svd(matrixA)

    return result


In [14]:
def image_blending(img_path, middle_homo,blend_sigma ):
    """
    :param img_path: all images
    :param middle_homo: middle homo 
    :return: blending result
    """
    tmp_image_list = []
    # blend sigma higher, then incoming image more vague
    blend_sigma = float(blend_sigma)
    for i in range(len(img_path)):
        im = cv2.imread(img_path[i])
        y_size, x_size = np.mgrid[0:im.shape[0], 0:im.shape[1]]
        im = np.dstack((im, np.exp(-((y_size - im.shape[0]/2) ** 2 + (x_size - im.shape[1]/2) ** 2) / (2.0 * blend_sigma ** 2)))) 
        tmp_image_list.append(image_transformation(im, np.array(middle_Homo[i]), cal_cornor(middle_Homo, img_path)[0]))
        

    Matrix1 = np.zeros(tmp_image_list[0].shape, dtype=float)
    Matrix2 = np.zeros(tmp_image_list[0].shape, dtype=float)
    Matrix2[:,:,3] = np.float(1.0)
    for i in range(len(tmp_image_list)):
        Matrix1[:,:,0] = Matrix1[:,:,0] + tmp_image_list[i][:,:,3] * tmp_image_list[i][:,:,0]
        Matrix1[:,:,1] = Matrix1[:,:,1] + tmp_image_list[i][:,:,3] * tmp_image_list[i][:,:,1]
        Matrix1[:,:,2] = Matrix1[:,:,2] + tmp_image_list[i][:,:,3] * tmp_image_list[i][:,:,2]
        Matrix1[:,:,3] = Matrix1[:,:,3] + tmp_image_list[i][:,:,3]
        
        Matrix2[:,:,0] =  Matrix2[:,:,0] + tmp_image_list[i][:,:,3]
        Matrix2[:,:,1] = Matrix2[:,:,1] + tmp_image_list[i][:,:,3]
        Matrix2[:,:,2] = Matrix2[:,:,2] + tmp_image_list[i][:,:,3]
        
    Matrix2[Matrix2 == 0] = 1
    return Matrix1/Matrix2

In [15]:
# img_1_left_path = "data\\part1\\Image1.jpg"
# img_1_right_path = "data\\part1\\Image2.jpg"
img_1_left_path = "data\\part1\\left.jpg"
img_1_right_path = "data\\part1\\right.jpg"

# hill
hill_left_path = "data\\part1\\hill\\1.jpg"
hill_mid_path = "data\\part1\\hill\\2.jpg"
hill_right_path ="data\\part1\\hill\\3.jpg"
# ledge
ledge_left_path = "data\\part1\\ledge\\1.jpg"
ledge_mid_path = "data\\part1\\ledge\\2.jpg"
ledge_part =  "data\\part1\\ledge\\stitched_image.jpg"
ledge_right_path = "data\\part1\\ledge\\3.jpg"
# pier
pier_left_path = "data\\part1\\pier\\1.jpg"
pier_mid_path = "data\\part1\\pier\\2.jpg"
pier_right_path = "data\\part1\\pier\\3.jpg"

# output folder path
out_path = "data\\part1\\output\\"




In [16]:
#workable for sample image left and right
# 0.7 and 0.8 very good

In [18]:
# path of all images
img_path = [ledge_left_path,ledge_mid_path, ledge_right_path]
homo_Matrix = [np.matrix(np.identity(3))]

for i in range(len(img_path) - 1):
    # read gray scale image first 
    img1 = cv2.imread(img_path[i],0)
    img2 = cv2.imread(img_path[i+1],0)
    # harris cornor detector
    harris_points2 = harris_cornor_detector(img2, num_of_points=500)
    harris_points1 = harris_cornor_detector(img1, num_of_points=500)

    # display feature points of first image
    # plt.imshow(img1,cmap='gray')
    # for points in harris_points1:
    #     plt.scatter(points[0], points[1])
    #     plt.draw()
    # plt.show()
    # 
    # # display feature points of second image
    # plt.imshow(img2,cmap='gray')
    # for points in harris_points2:
    #     plt.scatter(points[0], points[1])
    #     plt.draw()
    # plt.show()
    
    # extract neighbors as descriptors
    descriptor_1 = extract_decriptor(img1, harris_points1)
    descriptor_2 = extract_decriptor(img2, harris_points2)
    
    # we can adjust the threshold to have more or less matches
    initial_matches = get_match_from_dist(descriptor_1, 
                       descriptor_2,
                       threshold=0.7)
    
    print("num of initial_matches:", len(initial_matches))
    
    # we can adjust the tolerance for better model
    selected_points = np.matrix(np.hstack((harris_points1[initial_matches[:,0],0:2], harris_points2[initial_matches[:,1],0:2])))
    h = ransac(selected_points, tolerance=0.5, iteration=300)
    homo_Matrix.append(np.linalg.inv(h))



num of initial_matches: 115
Matches shape (115, 4)
number of homo inliers :

C:\Users\bluerain\PycharmProjects\cvmp2\new_venv\lib\site-packages\ipykernel_launcher.py:31: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


 24
residual: 8.026359160548154


num of initial_matches: 154
Matches shape (154, 4)
number of homo inliers : 27
residual: 1.3348454154824463


In [19]:
middle_Homo = cal_middle_homo(homo_Matrix, len(img_path))

result_img = image_blending(img_path, middle_Homo,blend_sigma = 100)

# print("panorama image:")
# plt.imshow(normalize(result_img))
# normalize(result_img),cmap='')
# plt.show()
cv2.imwrite(out_path+"stitched_image.jpg", result_img)



True